In [1]:
import os
os.chdir(path='../../')
import numpy as np
import scanpy as sc
import pandas as pd
from STForte import STGraph
from STForte import STForteModel
from STForte.helper import save_gdata
from STForte.helper import annotation_propagate
trial_name = "trial-DLPFC/multi_slides"

/home/wcx99075523/miniconda3/envs/stforte/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Global seed set to 0


In [2]:
from bokeh.themes import Theme
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.layouts import column, row
from bokeh.plotting import curdoc, figure, show
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, export_svg
import plotly.express as px
palette = px.colors.qualitative.Plotly
output_notebook()

Loading BokehJS ...

In [3]:
adata = sc.read_h5ad(f"./{trial_name}/outputs/stforte_multi.h5ad")
adata

AnnData object with n_obs × n_vars = 14364 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'spatialLIBD', 'section_id', 'STForte_Mask'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: '151673', '151674', '151675', '151676', 'hvg', 'log1p', 'pca'
    obsm: 'STForte_ATTR', 'STForte_COMB', 'STForte_TOPO', 'X_pca', 'X_pca_harmony', 'spatial'
    varm: 'PCs'
    layers: 'log1p'

In [4]:
adata_sp = sc.read_h5ad(f"./{trial_name}/outputs/sp_multi.h5ad")
adata_sp

AnnData object with n_obs × n_vars = 14364 × 84
    obs: 'spot_instance', 'section_id'
    obsm: 'SP_ATTR_FP', 'SP_COMB', 'SP_TOPO', 'spatial'

In [5]:
import numpy as np
from bokeh.models import Range1d

p_all = []
coord = adata_sp.obsm["spatial"][:, :2]
node_stat = adata_sp.obs["spot_instance"]
for ss in adata_sp.obs['section_id'].cat.categories:
    ss = f"{ss}"
    coord_sub = coord[adata_sp.obs["section_id"] == ss]
    stat_sub = node_stat[adata_sp.obs["section_id"] == ss]
    source = ColumnDataSource(dict(x=coord_sub[:, 0], y=coord_sub[:, 1],
                                   spot_instance=stat_sub))
    p = figure(title=f"Slice No. {ss}", width=240, height=240, toolbar_location=None, match_aspect=True)
    p.square(
        'x', 'y', source=source,
        fill_color=factor_cmap('spot_instance', palette=palette, factors=['Inferred', 'Observed']),
        angle=0, size=1, line_color=None
        )
    # p.y_range = Range1d(-12, 12)
    # p.x_range = Range1d(-11, 11)
    p_all.append(p)
grid = row(*p_all)
show(grid)

In [6]:
adata.write_h5ad(f"./{trial_name}/outputs/stforte_integrate.h5ad")
adata_sp.write_h5ad(f"./{trial_name}/outputs/sp_integrate.h5ad")